Cazyのサイトにある表に書かれている配列を取得する。

In [ ]:
#このセルを実行 -> アカウントを選択　-> 認証 してgoogleドライブをマウントする。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install biopython

In [ ]:
import os
import csv
import re
from Bio import SeqIO
from Bio import Entrez

メールアドレス、ファイル名を指定する。<br>
[配列の取得にメールアドレスが必要な理由](https://qiita.com/joemphilips/items/767c67524e4b7e328834)

In [ ]:
Entrez.email = "xxxx@yyy.zz"#メールアドレスを入れる。

#左側のフォルダマークをクリック。drive>MyDrive> から辿って、outdir= のパスを結果を出力したいディレクトリに書き換える。
#パスはフォルダ名の右側に出てくる点三つのマークをクリックしてコピーすると良い。
outdir='/content/drive/MyDrive/Colab_Notebooks/Qiita/fetch_cazy'#最後はスラッシュなし

cazy_url='http://www.cazy.org/GH96_characterized.html'#配列を取得したいCazyのページの url に書き換える
#characterized のページのみに対応している。

In [ ]:
#ウェブスクレイピングのためのライブラリをインポート
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
html = urlopen(cazy_url)

In [ ]:
bsObj = BeautifulSoup(html, "html.parser")
csvout = outdir + "/characterized.csv"
table = bsObj.findAll("table", {"class":"listing"})[0]
rows = table.findAll("tr")
#ページからテーブルを取得し、一旦csvで保存
with open(csvout, "w", encoding='utf-8') as file:
    writer = csv.writer(file)
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
        writer.writerow(csvRow)

In [ ]:
f1 = open(csvout, 'r')
reader = csv.reader(f1)
char_list = []

for row in reader:
    if len(row) >= 7:
        m = re.match("[A-Z]{3}[0-9]{5}(\.[0-9])?|[A-Z]{2}_[0-9]{4,10}(\.[0-9])?", row[4])#NCBI ID を検索
        if m:
            accno = m.group()
            char_list.append(accno)#NCBI ID をリストに格納
f1.close()

os.remove(csvout)#一旦保存した csv を削除。保持したい場合はこの行をコメントアウトする

In [ ]:
# このセルは配列数に比例して実行に時間がかかる
z = []
# Biopython の Entrez を用いて配列を取得し、リストに格納
for i in char_list:
    handle = Entrez.efetch(db="protein", id=i, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    z.append(record.format("fasta"))
handle.close()

In [ ]:
#urlからファイル名を生成する
dir_name = cazy_url.split('/')
regex = re.compile(r'html$')#urlの文末が.htmlで終わるか判定
m = regex.search(dir_name[-1])
if m:
  out = outdir + '/' + dir_name[-1].replace('.html', '.fasta')#TRUEなら拡張子を.fastaにしてファイル名に
else:
  dir_name2 = dir_name[-1].split('.html')#FALSEならurlをさらに分割…
  m2 = re.search('FUNC=[0-9]+',dir_name[-1])#FUNC=数字を検索して
  if m2:
    out = outdir + '/' + dir_name2[0] + str(m2.group().replace('FUNC=', '')) + '.fasta'#ファイル名を作成
  else:
     out = outdir + '/' + dir_name2[0] + '_unk_page.fasta'#FUNC=数字が見つからない場合のファイル名
print(out)

In [ ]:
#保存
with open(out, 'w') as r:
    for d in z:
        r.write("%s" % d)

In [ ]:
#保存したファイルを開いて、配列の数をカウント
a = 0
with open(out, "r") as f:
    lines = f.readlines()
    for i in lines:
        m = re.match('>', i)
        if m:
            a += 1
b = '取得した配列：' + str(a) + '個'
print(b)

配列の個数が正しいか確認（たまに取得できない配列があるため）